In [15]:
import openpyxl
import os
import json
import psycopg2
from config import config

In [16]:
def to_camel_case(input_string):
    words = input_string.split()

    camel_case_words = [words[0].lower()] + [word.capitalize() for word in words[1:]]

    return ''.join(camel_case_words)

In [17]:
def header_dict_util(sheet_obj):
    
    in_group = False
    header_row = None
    data_row = None
    file_name_lst = []
    header_dict = {}


    for row in sheet_obj.iter_rows(values_only=True):
        if any(row):  
            if not in_group:
                header_row = None
                data_row = None
                in_group = True
            if header_row is None:
                header_row = row
            elif data_row is None:
                data_row = row
                result_dict = dict(zip(header_row, data_row))
                file_name_lst.append(result_dict)
        elif in_group:
            in_group = False

    for i, result_dict in enumerate(file_name_lst, start=1):

        temp = {"title":result_dict["formTitle"], 
                "subTitle":result_dict["assetClassName"],
                "asset_class_code":result_dict["assetClassCode"],
                "asset_class_name":result_dict["assetClassName"],
                "plan_name":result_dict["planName"],
                "pm_title":result_dict["pmItem"]
               }

        header_dict[f'Group_{i}'] = temp
    
    return(header_dict)
    

In [23]:
def body_dict_util(sheet_obj):
    
    in_group = False
    current_group = None
    file_counter = 0
    body_components=[]
    body_components_dict={}



    for row in sheet_obj.iter_rows(values_only=True):

        if any(row):

            temp = {
                "type": "container",
                "key": "",
                "label": ""
                }

            if not in_group:

                in_group = True
                file_counter += 1

            if row[0] not in ['issueTitle','testTitle', 'tooltip', 'formTitle', 'assetClassName', 'assetClassCode', 'planName', 'pmItem']:
                
                print(row)
                temp['key'] = to_camel_case(row[1])
                temp['label'] = row[1]
                temp['title'] = row[0]

                body_components.append(temp)


        elif in_group:

            in_group = False
            body_components_dict[f'Group_{file_counter}'] = body_components
            body_components = []

    return body_components_dict
        

In [28]:
ref_json_file_path = '../src/ref_json.json'

with open(ref_json_file_path, 'rb') as json_file:
    json_schema = json.load(json_file)

src_file_path = "../src/configs-v2 (1).xlsx"
save_dir = "../sheets/"

In [29]:
wb_obj = openpyxl.load_workbook(src_file_path)

In [30]:
sheet_obj = wb_obj['ch.11']

In [31]:
company_id='22a8170a-f97c-432b-976a-28c4d7abf3ca'
header_dict = header_dict_util(sheet_obj)
body_components_dict = body_dict_util(sheet_obj)

('Dielectric Breakdown', 'Dielectric breakdown', None, 'NFPA 70B® Enhanced Fluid Sample Analysis Form', 'Oil-Filled Transformer', 'OFTR', '70B-ENHANCED', 'fluidSampleAnalysis', None, None, None, None)
('Acid Neutralization Number', 'Acid neutralization number', None, None, None, None, None, None, None, None, None, None)
('Specific Gravity', 'Specific gravity', None, None, None, None, None, None, None, None, None, None)
('Interfacial Tension', 'Interfacial tension', None, None, None, None, None, None, None, None, None, None)
('Color', 'Color', None, None, None, None, None, None, None, None, None, None)
('Visual Condition', 'Visual condition', None, None, None, None, None, None, None, None, None, None)
('Water Content', 'Water content', None, None, None, None, None, None, None, None, None, None)
('Power Factor', 'Power factor', None, None, None, None, None, None, None, None, None, None)
('Dissolved Gas Analysis', 'Dissolved gas analysis', None, None, None, None, None, None, None, None, N

In [32]:
header_dict

{'Group_1': {'title': 'NFPA 70B® Enhanced Fluid Sample Analysis Form',
  'subTitle': 'Oil-Filled Transformer',
  'asset_class_code': 'OFTR',
  'asset_class_name': 'Oil-Filled Transformer',
  'plan_name': '70B-ENHANCED',
  'pm_title': 'fluidSampleAnalysis'},
 'Group_2': {'title': 'NFPA 70B® Standard Fluid Sample Analysis Form',
  'subTitle': 'Oil-Filled Transformer',
  'asset_class_code': 'OFTR',
  'asset_class_name': 'Oil-Filled Transformer',
  'plan_name': '70B-STANDARD',
  'pm_title': 'fluidSampleAnalysis'},
 'Group_3': {'title': 'Basic Transformer Fluid Sample Analysis Form',
  'subTitle': 'Oil-Filled Transformer',
  'asset_class_code': 'OFTR',
  'asset_class_name': 'Oil-Filled Transformer',
  'plan_name': 'BASIC-XFMR',
  'pm_title': 'fluidSampleAnalysis'},
 'Group_4': {'title': 'NFPA 70B® Enhanced De-Energized Visual Inspection Form',
  'subTitle': 'Dry-Type Transformer (<= 600V)',
  'asset_class_code': 'DTTR-LV',
  'asset_class_name': 'Dry-Type Transformer (<= 600V)',
  'plan_name

In [33]:
for i in header_dict:
    
    json_schema['components'][0]['title'] = header_dict[i]['title']
    json_schema['components'][0]['subTitle'] = header_dict[i]['subTitle']
    
    json_schema['components'][1]['components'] = body_components_dict[i]
    
    json_schema['container'][1]['show'][0]['condition'] = "test.value" 
    json_schema['container'][2]['show'][0]['condition'] = "test.value"
    json_schema['container'][3]['show'][0]['condition'] = "test.value" 
    json_schema['container'][4]['show'][0]['condition'] = "test.value"
    
    new_json = json.dumps(json_schema)
    
    header_dict[i]['form_json']=new_json
    header_dict[i]['company_id']=company_id
    
    import uuid
    pmitemmasterform_id = uuid.uuid4()
    header_dict[i]['pmitemmasterform_id'] = pmitemmasterform_id
    
    
    print(header_dict[i])
    print('')
    
    print(new_json)
    print('----------------')

{'title': 'NFPA 70B® Enhanced Fluid Sample Analysis Form', 'subTitle': 'Oil-Filled Transformer', 'asset_class_code': 'OFTR', 'asset_class_name': 'Oil-Filled Transformer', 'plan_name': '70B-ENHANCED', 'pm_title': 'fluidSampleAnalysis', 'form_json': '{"display": "form", "components": [{"type": "header", "title": "NFPA 70B\\u00ae Enhanced Fluid Sample Analysis Form", "subTitle": "Oil-Filled Transformer", "components": [{"key": "temperature", "label": "Temperature", "type": "input", "inputType": "number", "suffix": "\\u00b0F"}, {"key": "humidity", "label": "Humidity", "type": "input", "inputType": "number", "suffix": "%"}]}, {"type": "body", "components": [{"type": "container", "key": "dielectricBreakdown", "label": "Dielectric breakdown", "title": "Dielectric Breakdown"}, {"type": "container", "key": "acidNeutralizationNumber", "label": "Acid neutralization number", "title": "Acid Neutralization Number"}, {"type": "container", "key": "specificGravity", "label": "Specific gravity", "title"

In [12]:
from db_service import addrecordsdb

In [13]:
addrecordsdb("PMItemMasterForms", header_dict['Group_1'])

Connecting to the PostgreSQL database...
45ef6004-39f6-4b8d-b233-93a1f901abf7
<class 'uuid.UUID'>
45ef6004-39f6-4b8d-b233-93a1f901abf7
<class 'str'>
Database connection closed.
